# Load environment variables

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

GRAPHRAG_API_KEY = os.getenv("GRAPHRAG_API_KEY")

OPENAI_API_MODEL = os.getenv("OPENAI_API_MODEL")
print(f"OPENAI_API_MODEL: {OPENAI_API_MODEL}")

OPENAI_API_EMBEDDING_MODEL = os.getenv("OPENAI_API_EMBEDDING_MODEL")
print(f"OPENAI_API_EMBEDDING_MODEL: {OPENAI_API_EMBEDDING_MODEL}")

FAISS_INDEX_NAME = os.getenv("FAISS_INDEX_NAME")
print(f"FAISS_INDEX_NAME: {FAISS_INDEX_NAME}")


OPENAI_API_MODEL: gpt-4o-2024-08-06
OPENAI_API_EMBEDDING_MODEL: text-embedding-3-large
FAISS_INDEX_NAME: HYBRID_FAISS_INDEX_NAME


# Load necessary libraries

In [3]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain.chains import GraphQAChain
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
import asyncio


# Initialize the llm and embeddings

In [4]:
# Initialize the graph transformer
llm = ChatOpenAI(
    model=OPENAI_API_MODEL,
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY,
)

# Initialize the embeddings
embeddings = OpenAIEmbeddings(
    model=OPENAI_API_EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY,
)

# Load the documents from the directory

In [5]:
# Load the documents with the type of files: .txt, .md, .pdf, .csv, .json
loader = DirectoryLoader("data", show_progress=True, use_multithreading=True)
documents = loader.load()

print(f"documents [0]: {documents[0]}")


  0%|          | 0/1 [00:00<?, ?it/s]/Users/mac/.pyenv/versions/3.11.7/envs/sofardatasai-rag-engine/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
The PDF <_io.BufferedReader name='data/multi-region-application-architecture.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
100%|██████████| 1/1 [00:13<00:00, 13.23s/it]

documents [0]: page_content='Multi-Region Application Architecture

AWS Implementation Guide

George Bearden

Eric Quinones

June 2020

Copyright (c) 2020 by Amazon.com, Inc. or its affiliates.

Multi-Region Application Architecture is licensed under the terms of the Apache License Version 2.0 available at https://www.apache.org/licenses/LICENSE-2.0

Amazon Web Services – Multi-Region Application Architecture

June 2020

Table of Contents

About This Guide .................................................................................................................. 3

Overview ................................................................................................................................... 3

Cost ....................................................................................................................................... 3

Architecture Overview .......................................................................................................... 4

S

In [6]:
# Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

print(f"split_documents [0]: {split_documents[0]}")

split_documents [0]: page_content='Multi-Region Application Architecture

AWS Implementation Guide

George Bearden

Eric Quinones

June 2020

Copyright (c) 2020 by Amazon.com, Inc. or its affiliates.

Multi-Region Application Architecture is licensed under the terms of the Apache License Version 2.0 available at https://www.apache.org/licenses/LICENSE-2.0

Amazon Web Services – Multi-Region Application Architecture

June 2020

Table of Contents

About This Guide .................................................................................................................. 3

Overview ................................................................................................................................... 3

Cost ....................................................................................................................................... 3

Architecture Overview .........................................................................................................

# Create the RetrievalQA chain



In [7]:
# Check if the index already exists
if os.path.exists(FAISS_INDEX_NAME):
    # Load the existing FAISS index
    vector_store = FAISS.load_local(FAISS_INDEX_NAME, embeddings, allow_dangerous_deserialization=True)
else:
    # Create a FAISS index from the text documents
    vector_store = FAISS.from_documents(split_documents, embeddings)

    # Save the FAISS index to disk
    FAISS.save_local(vector_store, FAISS_INDEX_NAME)

# Create the RetrievalQA chain
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    verbose=True,
)


# Create the GraphRAG using NetworkX Graphs and LangChain

In [8]:
llm_transformer = ChatOpenAI(
    model=OPENAI_API_MODEL,
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY,
)

# Define LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm_transformer)

# Define NetworkxEntityGraph
graph = NetworkxEntityGraph()

# Define the graphrag_ingest_data function

In [9]:
def graphrag_ingest_data(content: str):
    documents = [Document(page_content=content)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)

    for node in graph_documents[0].nodes:
        graph.add_node(node.id)

    for edge in graph_documents[0].relationships:
        graph._graph.add_edge(
            edge.source.id,
            edge.target.id,
            relation=edge.type
        )

        graph._graph.add_edge(
            edge.target.id,
            edge.source.id,
            relation=edge.type+" by",
        )

In [10]:
split_documents_length = len(split_documents)
print(f"split_documents_length: {split_documents_length}")

# print(f"split_documents[0]: {split_documents[0]}")
print(f"split_documents[1]: {split_documents[1].page_content}")

split_documents_length: 34
split_documents[1]: Architecture Overview .......................................................................................................... 4

Solution Components ............................................................................................................... 5

Application States ................................................................................................................. 5

Front-End Layer ................................................................................................................... 5

Routing Layer ....................................................................................................................... 6

Application Layer .................................................................................................................. 6

Key-Value Store Layer .......................................................................................................... 7


# Ingest the data into the graph

In [11]:
for document in split_documents:
    graphrag_ingest_data(document.page_content)


# Create the GraphQAChain

In [12]:
graph_qa_chain = GraphQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True
)

# Define the HybridRAG function

In [16]:
def hybrid_rag(query,generic_rag,graph_rag):
    print(f"query: {query}")

    # Doing generic RAG
    rag_result = generic_rag.run(query)

    # Printing the generic RAG Results
    print("----------------------------------------------")
    print("Generic RAG Result - ",rag_result)
    print("----------------------------------------------")

    # Doing GraphRAG
    graph_rag_result = graph_rag.run(query)

    # Printing the GraphRAG Results
    print("----------------------------------------------")
    print("Graph RAG Result - ",graph_rag_result)
    print("----------------------------------------------")

    prompt = f"""
    <Role>You are a helpful assistant, providing comprehensive answers to the questions based on the relevant context.</Role>
    <Context 1>{rag_result}</Context 1>
    <Context 2>{graph_rag_result}</Context 2>
    <Question>{query}</Question>
    <Answer>Answer:</Answer>
    """

    print(f"prompt: {prompt}")

    return llm(prompt),rag_result,graph_rag_result

# Entering the real query

In [17]:
real_query = 'What are the advantages of build Multi-Region Application Architecture on AWS Cloud?'

# Doing the HybridRAG

In [18]:
hybrid, rag, graphrag = hybrid_rag(real_query,retrieval_qa_chain,graph_qa_chain)

print("------------------------------------")
print("Hybrid RAG Result - ",hybrid.content)
print("------------------------------------")

query: What are the advantages of build Multi-Region Application Architecture on AWS Cloud?


> Entering new RetrievalQA chain...

> Finished chain.
----------------------------------------------
Generic RAG Result -  The advantages of building a Multi-Region Application Architecture on AWS Cloud include:

1. **Fault Tolerance**: The architecture is designed to handle regional failures with minimal impact on application availability. By replicating data and deploying applications across multiple regions, businesses can ensure continuity even if one region experiences an outage.

2. **Disaster Recovery**: The solution provides a mechanism for expedient recovery from regional failures with little-to-no application data loss. This is achieved through asynchronous data replication between regions.

3. **Improved Availability**: By deploying applications in multiple regions, businesses can achieve higher availability and reliability for their applications, ensuring that users can access ser